In [2]:
import pandas as pd
import numpy as np
from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

dfs = pd.read_excel("Acme.xlsx")

dfs_copy = dfs.copy()

X = dfs_copy[['click', 'cost', 'currently_insured', 'number_of_vehicles', 'number_of_drivers', 'marital_status', 'rank']]
y = dfs_copy['policies sold']

# we take out impression_id since it is just the index + 1. (in the "cols" variable next cell)

# what if #vehicles or #drivers is continuous?


dfs_copy.head(20)

,impression_id,click,cost,currently_insured,number_of_vehicles,number_of_drivers,marital_status,rank,policies sold
0,1,1,10,N,1,1,M,1,1
1,2,0,10,N,2,1,M,4,0
2,3,0,10,N,1,1,S,2,0
3,4,0,10,Y,2,1,M,5,0
4,5,0,10,Y,2,2,S,4,0
5,6,0,10,N,1,2,M,3,0
6,7,0,10,N,1,2,M,3,0
7,8,0,10,N,1,1,M,4,0
8,9,1,10,N,2,1,M,3,1
9,10,0,10,Y,1,1,M,2,0


In [3]:
cols = ['click','currently_insured','marital_status','number_of_vehicles','number_of_drivers','rank']
X[cols] = X[cols].astype(str)
dummies = pd.get_dummies(X[cols])

X_prime = pd.concat([X[[c for c in X.columns if c not in cols]],dummies],axis=1, sort=False)

c:\python3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_prime,y,test_size = .25,random_state = 614,shuffle = True,stratify = y)

X_train.head()

,cost,click_0,click_1,currently_insured_N,currently_insured_Y,marital_status_M,marital_status_S,number_of_vehicles_1,number_of_vehicles_2,number_of_drivers_1,number_of_drivers_2,rank_1,rank_2,rank_3,rank_4,rank_5
5651,10,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0
7900,10,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0
5938,10,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0
5210,10,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0
5309,10,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0


In [5]:
# first we'll try a logistic regression for policies sold.
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold

log_reg = LogisticRegression(max_iter = 10000)

# Cross-Validation on selecting one of the 30 features:
numfeatures = 15
cutoffs = range(100)

cv = StratifiedKFold(5, shuffle = True, random_state=614)

acc = np.empty((len(cutoffs)))    # acc = np.empty((numfeatures, len(cutoffs))), and so on
prec = np.empty((len(cutoffs)))
rec = np.empty((len(cutoffs)))
a = np.empty((5, len(cutoffs)))
p = np.empty((5, len(cutoffs)))
r = np.empty((5, len(cutoffs)))

#for i in range(numfeatures):                  #can't get this to work...
for cutoff in cutoffs:
        
    j=0
    for train_idx, test_idx in cv.split(X_train,y_train):

        X_train2 = X_train.iloc[train_idx]       # here train_idx and test_idx are indexes but second half
        y_train2 = y_train.iloc[train_idx]       # of equation is treating them as columns...try .iloc instead
        X_test2 = X_train.iloc[test_idx]
        y_test2 = y_train.iloc[test_idx]
        #X_train2[:,i]                          # this is the invalid bit
        clone_log_reg = clone(log_reg)
        clone_log_reg.fit(X_train2,y_train2.ravel())      # [:,i].reshape(-1,1)
        
        
        y_prob = clone_log_reg.predict_proba(X_test2)[:,1]      # [:,i].reshape(-1,1)
        y_predict = 1*(y_prob >= cutoff/100)
            
        a[j,cutoff] = 100*np.round(sum(y_predict == y_test2.ravel())/len(y_predict),5)      # a[i,j,cutoff]

        tp_train = np.sum(((y_predict == 1) & (y_test2 == 1)))
        fp_train = np.sum(((y_predict == 1) & (y_test2 == 0)))
        fn_train = np.sum(((y_predict == 0) & (y_test2 == 1)))
        tn_train = np.sum(((y_predict == 0) & (y_test2 == 0)))
            
        if tp_train + fp_train != 0:
            p[j,cutoff] = 100*np.round((tp_train/(tp_train + fp_train)),5)        # p[i,j,cutoff]
        else:
            p[j,cutoff] = 1
        if tp_train + fn_train != 0:
            r[j,cutoff] = 100*np.round((tp_train/(tp_train + fn_train)),5)
        else:
            r[j,cutoff] = 1
            
        j = j + 1
    
    acc[cutoff] = np.mean(a[:,cutoff])      #acc[i,cutoff], also a[i,:,cutoff]
    prec[cutoff] = np.mean(p[:,cutoff])
    rec[cutoff] = np.mean(r[:,cutoff])
        
        
# Here we find which cutoff maximizes accuracy,precision,&recall over all features (via mean):
    
j = 0
k = 0
l = 0
for cutoff in cutoffs[1:]:
    if j <= acc[cutoff]:        # np.mean(acc[:,cutoff])
        j = acc[cutoff]
    if k <= prec[cutoff]:
        k = prec[cutoff]
    if l <= rec[cutoff]:
        l = rec[cutoff]
        
finalcutoffacc = -1
finalcutoffprec = -1
finalcutoffrec = -1                      # a check to see if something goes wrong...

# Here we decide where our best cutoff is
for cutoff in cutoffs[1:]:
    if acc[cutoff] == j:         # np.mean(acc[:,cutoff])
        finalcutoffacc = cutoff
    if prec[cutoff] == k:
        finalcutoffprec = cutoff
    if rec[cutoff] == l:
        finalcutoffrec = cutoff


finalacc = acc[finalcutoffacc]         # acc[:,finalcutoffacc]
finalprec = prec[finalcutoffprec]
finalrec = rec[finalcutoffrec]
print(finalcutoffacc)                  # thus the best accuracy cutoff is at 0.49
print(finalcutoffprec)                  # thus the best precision cutoff is at 0.58
print(finalcutoffrec)                  # thus the best recall cutoff is at 0.14



49
58
14


In [9]:
# Here we reset the recall to use accuracy's cutoff, since any cutoff optimized for recall only will
# inevitably use the smallest cutoff to minimize FN and thus make TP/(TP+FN) as close to 1 as possible
finalrec = rec[finalcutoffacc]

print("accuracy:",finalacc,"%")
print("precision:",finalprec,"%")
print("recall:",finalrec,"%")

# recall and precision are awful here.... but we did use ALL features, maybe should go through subsets of them..

accuracy: 94.8134 %
precision: 59.769600000000004 %
recall: 33.5168 %


In [8]:
# first we'll try a logistic regression for policies sold.
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold

log_reg = LogisticRegression(max_iter = 10000)

# Cross-Validation on selecting one of the 30 features:
numfeatures = 15
cutoffs = range(100)

cv = StratifiedKFold(5, shuffle = True, random_state=614)

acc = np.empty((numfeatures, len(cutoffs)))
prec = np.empty((numfeatures, len(cutoffs)))
rec = np.empty((numfeatures, len(cutoffs)))
a = np.empty((numfeatures, 5, len(cutoffs)))
p = np.empty((numfeatures, 5, len(cutoffs)))
r = np.empty((numfeatures, 5, len(cutoffs)))

for i in range(numfeatures):                  #can't get this to work...
    for cutoff in cutoffs:
        
        j=0
        for train_idx, test_idx in cv.split(X_train,y_train):

            X_train2 = X_train.iloc[train_idx,]       # here train_idx and test_idx are indexes but second half
            y_train2 = y_train.iloc[train_idx]       # of equation is treating them as columns...try .iloc instead
            X_test2 = X_train.iloc[test_idx,]
            y_test2 = y_train.iloc[test_idx]
            #X_train2[:,i]                          # this is the invalid bit
            clone_log_reg = clone(log_reg)
            clone_log_reg.fit(X_train2[:,i].reshape(-1,1),y_train2.ravel())
        
        
            y_prob = clone_log_reg.predict_proba(X_test2[:,i].reshape(-1,1))[:,1]
            y_predict = 1*(y_prob >= cutoff/100)
            
            a[i,j,cutoff] = 100*np.round(sum(y_predict == y_test2.ravel())/len(y_predict),5)

            tp_train = np.sum(((y_predict == 1) & (y_test2 == 1)))
            fp_train = np.sum(((y_predict == 1) & (y_test2 == 0)))
            fn_train = np.sum(((y_predict == 0) & (y_test2 == 1)))
            tn_train = np.sum(((y_predict == 0) & (y_test2 == 0)))
            
            if tp_train + fp_train != 0:
                p[i,j,cutoff] = 100*np.round((tp_train/(tp_train + fp_train)),5)
            else:
                p[i,j,cutoff] = 1
            if tp_train + fn_train != 0:
                r[i,j,cutoff] = 100*np.round((tp_train/(tp_train + fn_train)),5)
            else:
                r[i,j,cutoff] = 1
            
            j = j + 1
    
        acc[i,cutoff] = np.mean(a[i,:,cutoff]) 
        prec[i,cutoff] = np.mean(p[i,:,cutoff])
        rec[i,cutoff] = np.mean(r[i,:,cutoff])
        
        
# Here we find which cutoff maximizes accuracy,precision,&recall over all features (via mean):
    
j = 0
k = 0
l = 0
for cutoff in cutoffs[1:]:
    if j <= np.mean(acc[:,cutoff]):       
        j = acc[cutoff]
    if k <= np.mean(prec[:,cutoff]):
        k = prec[cutoff]
    if l <= np.mean(rec[:,cutoff]):
        l = rec[cutoff]
        
finalcutoffacc = -1
finalcutoffprec = -1
finalcutoffrec = -1                      # a check to see if something goes wrong...

# Here we decide where our best cutoff is
for cutoff in cutoffs[1:]:
    if acc[cutoff] == j:         # np.mean(acc[:,cutoff])
        finalcutoffacc = cutoff
    if prec[cutoff] == k:
        finalcutoffprec = cutoff
    if rec[cutoff] == l:
        finalcutoffrec = cutoff


finalacc = acc[finalcutoffacc]         # acc[:,finalcutoffacc]
finalprec = prec[finalcutoffprec]
finalrec = rec[finalcutoffrec]
print(finalcutoffacc)                  # thus the best accuracy cutoff is at 0.49
print(finalcutoffprec)                  # thus the best precision cutoff is at 0.58
print(finalcutoffrec)                  # thus the best recall cutoff is at 0.14



TypeError: '(slice(None, None, None), 0)' is an invalid key

In [ ]:
# Here we reset the recall to use accuracy's cutoff, since any cutoff optimized for recall only will
# inevitably use the smallest cutoff to minimize FN and thus make TP/(TP+FN) as close to 1 as possible
finalrec = rec[finalcutoffacc]

print("accuracy:",finalacc,"%")
print("precision:",finalprec,"%")
print("recall:",finalrec,"%")

In [ ]:
# plots (for stuff with features)

plt.figure(figsize = (10,8))
plt.plot(finalacc,'o-')
plt.xlabel("Feature #",fontsize = 16)
plt.ylabel("Accuracy",fontsize = 16)

plt.figure(figsize = (10,8))
plt.plot(finalprec,'o-')
plt.xlabel("Feature #",fontsize = 16)
plt.ylabel("Precision",fontsize = 16)

plt.figure(figsize = (10,8))
plt.plot(finalrec,'o-')
plt.xlabel("Feature #",fontsize = 16)
plt.ylabel("Recall",fontsize = 16)

# Just to make sure precision =/= accuracy here:

plt.figure(figsize = (10,8))
plt.plot(finalprec-finalacc,'o-')
plt.xlabel("Feature #",fontsize = 16)
plt.ylabel("Precision - Accuracy",fontsize = 16)

plt.show()
